# 5교시 집계 연산

- 집계함수(Aggregate function) : 여러행 또는 테이블 전체 행으로부터 하나의 결과값을 반환하는 함수
- 그룹함수 (Group function) : 전체 행을 하나 이상의 컬럼을 기준으로 컬럼값에 따라 그룹화하여 그룹별로 결과를 출력하는 함수

### 목차
* [1. 집계 함수](#1.-집계-함수)
* [2. 그룹 함수](#2.-그룹-함수)
* [참고자료](#참고자료)


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

22/04/12 19:38:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
""" 구매 이력 데이터 """
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/all")
    .coalesce(5)
)
df.cache()
df.createOrReplaceTempView("dfTable")

In [3]:
df.show(5, truncate=False)
df.count()

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
+---------+---------+-----------------------------------

541909

## 1. 집계 함수
### 1.1 로우 수 (count, countDistinct, approx_count_distinct)

In [4]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
from pyspark.sql.functions import *

df.selectExpr("count(*)").show()
df.where("Description is null").selectExpr("count(1)").show() # 1,454
df.selectExpr("count(Description)").show() # 540,455 + 1,454 = 541,909

+--------+
|count(1)|
+--------+
|  541909|
+--------+

+--------+
|count(1)|
+--------+
|    1454|
+--------+

+------------------+
|count(Description)|
+------------------+
|            540455|
+------------------+



In [6]:
from pyspark.sql.functions import *
# 명시적으로 컬럼을 지정한 경우 해당 컬럼이 널 인 경우 해당 로우는 제외됩니다
df.select(countDistinct("StockCode")).show()
df.selectExpr("count(distinct StockCode)").show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [7]:
from pyspark.sql.functions import *
# 근사치로 구하지만 연산 속도가 빠름
df.select(approx_count_distinct("StockCode", 0.1)).show() # 0.1은 최대 추정 오류율
df.select(approx_count_distinct("StockCode", 0.01)).show() 

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



22/04/12 19:23:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            4079|
+--------------------------------+



### 1.2 수치 집계 함수 (first, last, min, max, sum, sumDistinct, avg)

In [8]:
from pyspark.sql.functions import *
df.select(first("StockCode"), last("StockCode")).show(1) # null도 감안하려면 True

df.select(min("Quantity"), max("Quantity")).show(1)
df.select(min("Description"), max("Description")).show(1) # 문자열

df.select(sum("Quantity")).show(1)
df.select(sumDistinct("Quantity")).show(1) # 고유값을 합산

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+

+--------------------+-----------------+
|    min(Description)| max(Description)|
+--------------------+-----------------+
| 4 PURPLE FLOCK D...|wrongly sold sets|
+--------------------+-----------------+

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



### 1.3 통계 집계 함수 (avg, mean, variance, stddev) 

In [9]:
from pyspark.sql.functions import *

df.select(
    count("Quantity").alias("total_transcations"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_transcations"),    
).selectExpr(
    "total_purchases / total_transcations",
    "avg_purchases",
    "mean_transcations").show(3)

+--------------------------------------+----------------+-----------------+
|(total_purchases / total_transcations)|   avg_purchases|mean_transcations|
+--------------------------------------+----------------+-----------------+
|                      9.55224954743324|9.55224954743324| 9.55224954743324|
+--------------------------------------+----------------+-----------------+



* 표본표준분산 및 편차: variance, stddev
* 모표준분산 및 편차 : var_pop, stddev_pop

In [10]:
df.select(
    variance("Quantity")
    , stddev("Quantity")
    , var_samp("Quantity")
    , stddev_samp("Quantity")
    , var_pop("Quantity")
    , stddev_pop("Quantity")
).show()

+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_samp(Quantity)|stddev_samp(Quantity)|var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
| 47559.39140929892|   218.08115785023455| 47559.39140929892|   218.08115785023455|47559.30364660923|  218.08095663447835|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+



#### 1.3.1 분산과 표준편차
+ 표본표준분산 및 편차: variance, stddev
+ 모표준분산 및 편차 : var_pop, stddev_pop

In [11]:
from pyspark.sql.functions import variance, stddev
from pyspark.sql.functions import var_samp, stddev_samp
from pyspark.sql.functions import var_pop, stddev_pop

df.select(variance("Quantity"), stddev("Quantity"),      
          var_samp("Quantity"), stddev_samp("Quantity"), # 위와 동일
          var_pop("Quantity"), stddev_pop("Quantity")).show()

+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_samp(Quantity)|stddev_samp(Quantity)|var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
| 47559.39140929892|   218.08115785023455| 47559.39140929892|   218.08115785023455|47559.30364660923|  218.08095663447835|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+



In [12]:
spark.createDataFrame(df.select("*").take(1)).select(variance("Quantity"), stddev("Quantity"),      
          var_samp("Quantity"), stddev_samp("Quantity"), # 위와 동일
          var_pop("Quantity"), stddev_pop("Quantity")).show() # 1일 때는 NaN이 나옵니다.

+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_samp(Quantity)|stddev_samp(Quantity)|var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+
|              null|                 null|              null|                 null|              0.0|                 0.0|
+------------------+---------------------+------------------+---------------------+-----------------+--------------------+



#### 1.3.2 비대칭도와 첨도
+ 비대칭도와 첨도 : https://www.youtube.com/watch?time_continue=2&v=g9VOhfy2WWY

In [13]:
from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610528376|119768.05495530753|
+--------------------+------------------+



![왜도](https://www.oppadu.com/wp-content/uploads/2021/02/%EC%97%91%EC%85%80-skew-%ED%95%A8%EC%88%98-%EC%99%9C%EB%8F%84-%EC%84%A4%EB%AA%85-600x246.png)

#### 비대칭도, 왜도 (skewness)
> 왜도는 데이터가 대칭이 아닌 정도, 비대칭도입니다. 왜도 값(0, 양수 또는 음수)이 데이터 형상에 대한 정보를 나타냅니다.
데이터가 대칭에 가까울수록 왜도 값이 0에 근접합니다. 그러나 왜도 부족만으로 정규성을 의미하지는 않습니다.

#### 첨도(kurtosis)
> 첨도는 분포의 꼬리가 정규 분포와 어떻게 다른지 나타냅니다. 완전히 정규 분포를 따르는 데이터의 첨도 값은 0입니다.
분포의 첨도 값이 양수이면 분포의 꼬리가 정규 분포보다 두껍다는 것을 나타냅니다
분포의 첨도 값이 음수이면 분포의 꼬리가 정규 분포보다 얇다는 것을 나타냅니다. 

#### [skewness](https://github.com/apache/spark/blob/5a7403623d0525c23ab8ae575e9d1383e3e10635/sql/catalyst/src/main/scala/org/apache/spark/sql/catalyst/expressions/aggregate/CentralMomentAgg.scala#L231)
```scala
org.apache.spark.sql.catalyst.expressions.aggregate.CentralMomentAgg

def skewness(columnName: String): Column = skewness(Column(columnName))
def skewness(e: Column): Column = withAggregateFunction { Skewness(e.expr) }
def kurtosis(e: Column): Column = withAggregateFunction { Kurtosis(e.expr) }
def kurtosis(columnName: String): Column = kurtosis(Column(columnName))
```

#### 1.3.3 공분산과 상관관계
+ 표본공분산(cover_samp), 모공분산(cover_pop)

In [14]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

df.select(corr("InvoiceNo", "Quantity"), covar_pop("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity")).show()

+-------------------------+------------------------------+-------------------------------+
|corr(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|
+-------------------------+------------------------------+-------------------------------+
|     4.912186085648426E-4|            1052.7260778770628|              1052.728054393167|
+-------------------------+------------------------------+-------------------------------+




          
#### [공분산](https://ko.wikipedia.org/wiki/%EA%B3%B5%EB%B6%84%EC%82%B0) (covariance)
> 공분산(共分散, 영어: covariance)은 2개의 확률변수의 상관정도를 나타내는 값이다.(1개의 변수의 이산정도를 나타내는 분산과는 별개임) 만약 2개의 변수중 하나의 값이 상승하는 경향을 보일 때, 다른 값도 상승하는 경향의 상관관계에 있다면, 공분산의 값은 양수가 될 것이다. 반대로 2개의 변수중 하나의 값이 상승하는 경향을 보일 때, 다른 값이 하강하는 경향을 보인다면 공분산의 값은 음수가 된다. <br>
<br>

<br>
단, 100점만점인 두과목의 점수 공분산은 별로 상관성이 부족하지만 100점만점이기 때문에 큰 값이 나오고
10점짜리 두과목의 점수 공분산은 상관성이 아주 높을지만 10점만점이기 때문에 작은값이 나온다


#### [상관관계](https://ko.wikipedia.org/wiki/%EC%83%81%EA%B4%80_%EB%B6%84%EC%84%9D) (correlation)
> 상관 분석(Correlation analysis)은 확률론과 통계학에서 두 변수간에 어떤 선형적 관계를 갖고 있는 지를 분석하는 방법이다. 
![](https://t1.daumcdn.net/cfile/tistory/99E3B03C5DF5EF520B)

#### 피어슨 상관 계수
> 피어슨 상관 계수란 두 변수 X 와 Y 간의 선형 상관 관계를 계량화한 수치다 . 피어슨 상관 계수는 코시-슈바르츠 부등식에 의해 +1과 -1 사이의 값을 가지며, +1은 완벽한 양의 선형 상관 관계, 0은 선형 상관 관계 없음, -1은 완벽한 음의 선형 상관 관계를 의미한다.

#### Perason's r = X와 Y가 함께 변하는 정도 / X와 Y가 각각 변하는 정도
##### r 값은 X 와 Y 가 완전히 동일하면 +1, 전혀 다르면 0, 반대방향으로 완전히 동일 하면 –1 을 가진다.

### 1.4 복합 데이터 타입의 집계

In [14]:
from pyspark.sql.functions import collect_list, collect_set, size

df.select(collect_list("Country"), collect_set("Country")).show()

+---------------------+--------------------+
|collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
| [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+



In [15]:
df.select(size(collect_list("Country")), size(collect_set("Country"))).show() # 각 컬럼의 복합데이터 사이즈

+---------------------------+--------------------------+
|size(collect_list(Country))|size(collect_set(Country))|
+---------------------------+--------------------------+
|                     541909|                        38|
+---------------------------+--------------------------+



In [16]:
df.select(countDistinct("Country")).show() # 중복없이 카운트

+-----------------------+
|count(DISTINCT Country)|
+-----------------------+
|                     38|
+-----------------------+



## 2. 그룹 함수

### 2.1 표현식을 이용한 그룹화

In [18]:
from pyspark.sql.functions import count
df.printSchema()
df.groupBy("InvoiceNo", "CustomerId").agg(expr("count(Quantity) as CountOfQuantity")).show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



+---------+----------+---------------+
|InvoiceNo|CustomerId|CountOfQuantity|
+---------+----------+---------------+
|   536366|     17850|              2|
|   536367|     13047|             12|
|   536369|     13047|              1|
|   536376|     15291|              2|
|   536387|     16029|              5|
+---------+----------+---------------+
only showing top 5 rows



### 2.2 맵을 이용한 그룹화
> 파이선의 딕셔너리 데이터 타입을 활용하여 집계함수의 표현이 가능 

In [19]:
df.groupBy("InvoiceNo").agg( { "Quantity" : "avg", "UnitPrice" : "stddev_pop" } ).show(5)

+---------+---------------------+------------------+
|InvoiceNo|stddev_pop(UnitPrice)|     avg(Quantity)|
+---------+---------------------+------------------+
|   536370|   3.6916533897428674|             22.45|
|   536380|                  0.0|              24.0|
|   536384|   3.5529802474898813|14.615384615384615|
|   536387|   1.0775602071346178|             288.0|
|   536397|                  0.0|              30.0|
+---------+---------------------+------------------+
only showing top 5 rows



### [참고] 3. UDAF
+ User Defined Aggregation Function, 
+ UDAF를 생성하려면 기본 클래스인 UserDefinedAggregateFunction을 상속
+ UDAF는 현재 스칼라와 자바로만 사용할 수 있음(ver 2.3)

※ Efficient UD(A)Fs with PySpark https://www.inovex.de/blog/efficient-udafs-with-pyspark/

### <font color=blue>1. [중급]</font> 구매 이력 CSV f"{work_data}/retail-data/all" 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 상품코드(StockCode)의 유일한 값의 갯수를 출력하세요
#### 4. 상품단가(UnitPrice)의 최소, 최대 값을 출력하세요
#### 5. 송장번호(StockCode)별로 송장별총매출금액(TotalInvoicePrice)를 계산하고 내림차순으로 정렬하세요
#### 6. 송장별총매출금액(TotalInvoicePrice)이 최고금액이 송장을 필터하여 검증해 보세요
##### 예를 들어 `select sum(unit-price * quantity) from table where invoiceno = '123456'` 와 같은 쿼리로 검증이 가능합니다

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df1 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/retail-data/all")
)
df1.printSchema()
df1.show()
answer = df1.withColumn("TotalPrice", expr("UnitPrice * Quantity")).groupBy("InvoiceNo").agg(sum("TotalPrice").alias("TotalInvoicePrice"))
answer.printSchema()
display(answer.orderBy(desc("TotalInvoicePrice")).limit(10))

df1.where("InvoiceNo = '581483'").select(sum(expr("UnitPrice * Quantity"))).show()
```

</details>


In [1]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=green>2. [기본]</font> 매출 테이블 f"{work_data}/tbl_purchase.csv" CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 제품(p_name)별 금액(p_amount) 의 전체 합인 총 매출금액(sum_amount)을 구하세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df2 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
df2.printSchema()
df2.show()
answer = df2.groupBy("p_name").agg(sum("p_amount").alias("sum_amount"))
answer.printSchema()
display(answer)

```

</details>


In [2]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=green>3. [기본]</font> 매출 테이블 f"{work_data}/tbl_purchase.csv" CSV 파일을 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터 10건을 출력하세요
#### 3. 구매 금액의 합이 가장 높은 고객(p_uid)을 구하세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df3 = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{work_data}/tbl_purchase.csv")
)
df3.printSchema()
df3.show()
answer = df2.groupBy("p_uid").agg(sum("p_amount").alias("sum_amount_per_user"))
answer.printSchema()
display(answer)
```

</details>


In [3]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

### <font color=red>4. [고급]</font> 샌프란시스코 긴급출동 데이터 CSV 파일인 f"{work_data}/learning-spark/sf-fire-calls.csv"를 읽고
#### 1. 스키마를 출력하세요
#### 2. 데이터를 3건 출력하세요
#### 3. 호출의 종류(CallType)가 어떤 것들이 있는지 출력하세요 (중복제거)
#### 3. 샌프란시스코에서 발생의 가장 빈도수가 높은 종류(CallType)를 구하고 빈도수를 구하세요
#### 4. 샌프란시스코에서 발생하는 최고 빈도수 3건은 무엇인가요? 

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
df3 = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(f"{work_data}/learning-spark/sf-fire-calls.csv")
)
df3.printSchema()
df3.show(3)
df3.createOrReplaceTempView("fire_calls")
spark.sql("select distinct(CallType) from fire_calls").show(truncate=False)

answer = spark.sql("select CallType, count(CallType) as CallTypeCount from fire_calls group by CallType order by CallTypeCount desc")
display(answer.limit(3))
```

</details>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

## 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
#### 4. [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
#### 5. [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)